In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as st

import json
import tqdm

from pbpstats.client import Client

In [2]:
# want to use the data_nba provider
settings = {
    "Games": {"source": "web", "data_provider": "data_nba"},
    "Possessions": {"source": "web", "data_provider": "data_nba"}
}
client = Client(settings)
season = client.Season("nba", "2021-22", "Regular Season")

In [3]:
# let's find all the lakers games

lakers_games = [g for g in season.games.final_games if g['home_team_abbreviation']=='LAL'                                                     or g['away_team_abbreviation']=='LAL']
laker_games_df = pd.DataFrame.from_records(lakers_games)
laker_games_df.to_csv('./all_lakers_games.csv', index = False)
laker_games_df.head()

,game_id,date,status,home_team_id,home_team_abbreviation,home_score,away_team_id,away_team_abbreviation,away_score
0,0022100002,2021-10-19,Final,1610612747,LAL,114,1610612744,GSW,121
1,0022100025,2021-10-22,Final,1610612747,LAL,105,1610612756,PHX,115
2,0022100040,2021-10-24,Final,1610612747,LAL,121,1610612763,MEM,118
3,0022100053,2021-10-26,Final,1610612759,SAS,121,1610612747,LAL,125
4,0022100061,2021-10-27,Final,1610612760,OKC,123,1610612747,LAL,115


In [4]:
# let's get PBP stats for each game

gid = list(laker_games_df['game_id'])[0]
game = client.Game(gid)

In [5]:
# let's build a possession data parser
# each possession needs a:
#     - Game ID
#     - Period (1-4, OT)
#     - Start time
#     - End time
#     - Offensive Team ID
#     - Up to 10 Events
#         - Event Type
#         - Time
#         - Event Number
#     - Timeout in the possession?
#     - Possession start type
#     - Possession stats
#     - Previous Possession
#         - Ending event
#         - Rebounder who ended it
#         - Shooter who ended it
#         - Stealer who ended it
#         - Turnover who ended it
#         - Timeout in the prior possession? 
#     - Next Possession

# for i in game.possessions.items:
#     print(len(i.data['events']))
i = game.possessions.items[4]
i.previous_possession.number

4

In [6]:
player_ids = pd.read_csv('./NBA_Player_IDs.csv', encoding= 'unicode_escape')
player_ids.head()

,BBRefName,BBRefLink,BBRefID,BBRefBirthDate,NBAName,NBALink,NBAID,NBABirthDate,ESPNName,ESPNLink,ESPNID,ESPNBirthDate,SpotracName,SpotracLink,SpotracID
0,A.J. Hammons,https://www.basketball-reference.com/players/h...,hammoaj01,8/27/1992,AJ Hammons,https://stats.nba.com/player/1627773/,1627773.0,8/27/1992,AJ Hammons,http://www.espn.com/nba/player/_/id/2991178/aj...,2991178.0,8/27/1992,A.J. Hammons,https://www.spotrac.com/redirect/player/20252/,20252.0
1,A.J. Price,https://www.basketball-reference.com/players/p...,priceaj01,10/7/1986,AJ Price,https://stats.nba.com/player/201985/,201985.0,10/7/1986,A.J. Price,http://www.espn.com/nba/player/_/id/4010/aj-price,4010.0,10/7/1986,A.J. Price,https://www.spotrac.com/redirect/player/6292/,6292.0
2,Aaron Brooks,https://www.basketball-reference.com/players/b...,brookaa01,1/14/1985,Aaron Brooks,https://stats.nba.com/player/201166/,201166.0,1/14/1985,Aaron Brooks,http://www.espn.com/nba/player/_/id/3192/aaron...,3192.0,1/14/1985,Aaron Brooks,https://www.spotrac.com/redirect/player/2390/,2390.0
3,Aaron Gordon,https://www.basketball-reference.com/players/g...,gordoaa01,9/16/1995,Aaron Gordon,https://stats.nba.com/player/203932/,203932.0,9/16/1995,Aaron Gordon,http://www.espn.com/nba/player/_/id/3064290/aa...,3064290.0,9/16/1995,Aaron Gordon,https://www.spotrac.com/redirect/player/15356/,15356.0
4,Aaron Gray,https://www.basketball-reference.com/players/g...,grayaa01,12/7/1984,Aaron Gray,https://stats.nba.com/player/201189/,201189.0,12/7/1984,Aaron Gray,http://www.espn.com/nba/player/_/id/3207/aaron...,3207.0,12/7/1984,Aaron Gray,https://www.spotrac.com/redirect/player/2244/,2244.0


In [7]:
def player_id_to_name(i):
    d = player_ids[player_ids['NBAID']==i]
    if len(d)==1:
        return list(d['NBAName'])[0]
    else:
        return None

In [8]:
def parse_possession_item(i):
    item_dict_1 = {
        'game_id'    : i.game_id,
        'period'     : i.period,
        'pos_number' : i.number,
        'start_time' : i.start_time,
        'end_time'   : i.end_time,
        'offense_id' : i.offense_team_id, 
    }

    for j in range(len(i.events)):
        item_dict_1['event'+str(j).zfill(2)+'_time']   = i.events[j].clock
        item_dict_1['event'+str(j).zfill(2)+'_type']   = type(i.events[j]).__name__[4:] # trim "Data" prefix
        item_dict_1['event'+str(j).zfill(2)+'_player'] = player_id_to_name(i.events[j].player1_id)

    for j in range(len(i.events), 10): # pad to 10 events
        item_dict_1['event'+str(j).zfill(2)+'_time']   = None
        item_dict_1['event'+str(j).zfill(2)+'_type']   = None
        item_dict_1['event'+str(j).zfill(2)+'_player'] = None

    item_dict_2 = {
        'pos_has_timeout'       : i.possession_has_timeout,
        'pos_start_type'        : i.possession_start_type,
        'prev_pos'              : i.previous_possession.number if i.previous_possession is not None else -1,
        #'prev_pos_ending_event' : i.previous_possession_ending_event,
        'prev_reb_player'       : player_id_to_name(i.previous_possession_end_rebound_player_id),
        'prev_sht_player'       : player_id_to_name(i.previous_possession_end_shooter_player_id),
        'prev_stl_player'       : player_id_to_name(i.previous_possession_end_steal_player_id),
        'prev_tov_player'       : player_id_to_name(i.previous_possession_end_turnover_player_id),
        'prev_has_timeout'      : i.previous_possession_has_timeout,
        'next_pos'              : i.next_possession.number if i.next_possession is not None else -1
    }

    return item_dict_1 | item_dict_2

In [9]:
%%time

all_pos_json = [
    parse_possession_item(i) for i in game.possessions.items
]

all_pos_df = pd.DataFrame.from_records([j for j in all_pos_json if j is not None])
all_pos_df

CPU times: user 367 ms, sys: 2.76 ms, total: 370 ms
Wall time: 370 ms


,game_id,period,pos_number,start_time,end_time,offense_id,event00_time,event00_type,event00_player,event01_time,...,event09_player,pos_has_timeout,pos_start_type,prev_pos,prev_reb_player,prev_sht_player,prev_stl_player,prev_tov_player,prev_has_timeout,next_pos
0,0022100002,1,1,12:00,11:53,1610612747,12:00,StartOfPeriod,None,11:57,...,None,False,OffDeadball,-1,None,None,None,None,False,2
1,0022100002,1,2,11:53,11:37,1610612744,11:37,FieldGoal,Kevon Looney,None,...,None,False,OffAtRimMake,1,None,DeAndre Jordan,None,None,False,3
2,0022100002,1,3,11:37,11:24,1610612747,11:24,Turnover,LeBron James,None,...,None,False,OffAtRimMake,2,None,Kevon Looney,None,None,False,4
3,0022100002,1,4,11:24,11:18,1610612744,11:24,Foul,DeAndre Jordan,11:18,...,None,False,OffLiveBallTurnover,3,None,None,Stephen Curry,LeBron James,False,5
4,0022100002,1,5,11:18,11:02,1610612747,11:05,FieldGoal,Anthony Davis,11:02,...,None,False,OffLongMidRangeMake,4,None,Jordan Poole,None,None,False,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,0022100002,4,46,00:28.3,00:25.1,1610612744,00:25.1,Foul,Avery Bradley,00:25.1,...,None,False,OffArc3Make,45,None,Anthony Davis,None,None,False,47
221,0022100002,4,47,00:25.1,00:18.6,1610612747,00:18.6,FieldGoal,Russell Westbrook,None,...,None,False,OffFTMake,46,None,None,None,None,False,48
222,0022100002,4,48,00:18.6,00:11.9,1610612744,00:11.9,Foul,Kent Bazemore,00:11.9,...,None,False,OffAtRimMake,47,None,Russell Westbrook,None,None,False,49
223,0022100002,4,49,00:11.9,00:07.0,1610612747,00:07.0,FieldGoal,Malik Monk,None,...,None,False,OffFTMake,48,None,None,None,None,False,50


In [10]:
def get_possession_df(game_id):
    try:
        g = client.Game(game_id)
        return pd.DataFrame.from_records([parse_possession_item(i) for i in g.possessions.items])
    except:
        return None

In [11]:
all_pos_list = [get_possession_df(g_id) for g_id in tqdm.tqdm_notebook(list(laker_games_df['game_id']))]
all_pos_list = [a for a in all_pos_list if a is not None]
all_pos_df = pd.concat(all_pos_list).reset_index()

all_pos_df

/var/folders/dc/w29p98q54dv6v4wg20gnvzg40000gn/T/ipykernel_66735/797942961.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  all_pos_list = [get_possession_df(g_id) for g_id in tqdm.tqdm_notebook(list(laker_games_df['game_id']))]


  0%|          | 0/37 [00:00<?, ?it/s]

,index,game_id,period,pos_number,start_time,end_time,offense_id,event00_time,event00_type,event00_player,...,event14_player,event15_time,event15_type,event15_player,event16_time,event16_type,event16_player,event17_time,event17_type,event17_player
0,0,0022100002,1,1,12:00,11:53,1610612747,12:00,StartOfPeriod,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0022100002,1,2,11:53,11:37,1610612744,11:37,FieldGoal,Kevon Looney,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0022100002,1,3,11:37,11:24,1610612747,11:24,Turnover,LeBron James,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0022100002,1,4,11:24,11:18,1610612744,11:24,Foul,DeAndre Jordan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0022100002,1,5,11:18,11:02,1610612747,11:05,FieldGoal,Anthony Davis,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7411,195,0022100538,4,43,01:06,00:51.4,1610612747,00:53.5,FieldGoal,Kent Bazemore,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7412,196,0022100538,4,44,00:51.4,00:33.7,1610612757,00:35.7,FieldGoal,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7413,197,0022100538,4,45,00:33.7,00:13.6,1610612747,00:13.6,FieldGoal,Kent Bazemore,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7414,198,0022100538,4,46,00:13.6,00:00.1,1610612757,00:02.6,FieldGoal,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
all_pos_df[(all_pos_df['event00_type']=='FieldGoal')&([x is None for x in list(all_pos_df['event00_player'])])]

,index,game_id,period,pos_number,start_time,end_time,offense_id,event00_time,event00_type,event00_player,...,event14_player,event15_time,event15_type,event15_player,event16_time,event16_type,event16_player,event17_time,event17_type,event17_player
78,78,0022100002,2,23,07:23,07:17,1610612744,07:19,FieldGoal,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,96,0022100002,2,41,04:21,04:13,1610612744,04:17,FieldGoal,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
379,154,0022100025,4,3,11:19,11:04,1610612747,11:04,FieldGoal,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
397,172,0022100025,4,21,06:12,06:01,1610612747,06:01,FieldGoal,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
413,188,0022100025,4,37,01:55,01:51,1610612747,01:51,FieldGoal,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7406,190,0022100538,4,38,02:33,02:12,1610612757,02:17,FieldGoal,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7408,192,0022100538,4,40,02:05,01:45,1610612757,01:45,FieldGoal,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7409,193,0022100538,4,41,01:45,01:24,1610612747,01:24,FieldGoal,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7412,196,0022100538,4,44,00:51.4,00:33.7,1610612757,00:35.7,FieldGoal,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
all_pos_df.to_csv('./all_lakers_possessions_20211228.csv', index=False)